# Setup and Load dataset



In [ ]:
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2e5059e1abcd1fb1f653800d8423606be083b86bb2721db1535cc6d16eb21d56
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 15hmSSQBP0hPCJHrQRBjyfqd_bPU29Rwr

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=15hmSSQBP0hPCJHrQRBjyfqd_bPU29Rwr
To: /content/MIND.zip
100% 44.4M/44.4M [00:00<00:00, 174MB/s]


In [ ]:
!unzip -o "MIND.zip"  -d  "/content"

Archive:  MIND.zip
  inflating: /content/MIND/behaviors.tsv  
  inflating: /content/MIND/news.tsv  


## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="MIND/news.tsv"

###### Run below cell

In [ ]:
content_df  = pd.read_table( 'MIND/news.tsv',
            names=['newid', 'vertical', 'subvertical', 'title',
                            'abstract', 'url', 'entities in title', 'entities in abstract'],
                     usecols = ['newid','vertical', 'subvertical', 'title', 'abstract'])

In [ ]:
content_df=content_df

## Details about dataset

In [ ]:
itemid="newid"

In [ ]:
features=['abstract']

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [ ]:
stopwords=set(stopwords.words('english'))

In [ ]:
def tokenizer(sentence, min_words=4, max_words=200, stopwords=stopwords, lemmatize=True):
    
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

In [ ]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

# Model train

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')


In [ ]:
corpus_embeddings = model.encode(content_df.clean.values,show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/1603 [00:00<?, ?it/s]

# Annoy

In [ ]:
!pip install annoy==1.17.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=582796 sha256=bbe30cf0cc07ded4e0749512d4b7d2d706d7fdddf76491e17dedca31b2cf53f7
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [ ]:

from annoy import AnnoyIndex
import torch

In [ ]:

embedding_size=corpus_embeddings.shape[1]
n_trees=100
annoy_index = AnnoyIndex(embedding_size, 'angular')

for i in range(len(corpus_embeddings)):
        annoy_index.add_item(i, corpus_embeddings[i])

annoy_index.build(n_trees)


True

In [ ]:
annoy_index_path='annoy_index'
annoy_index.save(annoy_index_path)

True

In [ ]:
torch_corpus_embeddings = torch.from_numpy(corpus_embeddings)

In [ ]:
query="Shop the notebooks, jackets"
query_embedding = model.encode(query, convert_to_tensor=True)


In [ ]:
TOP_K=10
corpus_ids, scores = annoy_index.get_nns_by_vector(query_embedding, TOP_K, include_distances=True)
annoy_hits = []
for id, score in zip(corpus_ids, scores):
    annoy_hits.append({'corpus_id': id, 'score': 1-((score**2) / 2)})

In [ ]:
annoy_hit_corpus=[]
print("Results:")
for hit in annoy_hits[0:TOP_K]:
        print("\t{}".format( content_df.clean.values[hit['corpus_id']]))
        annoy_hit_corpus.append(hit['corpus_id'])



Results:
	 shop the notebooks  jackets  and more that the royals can t live without 
	 you ll see lots of laptop deals in the coming weeks  doorbusters for notebooks under  500  maybe under  400 or  300 if you re lucky  and while there  no shame in putting on three layers in the cold on black friday morning only to elbow someone in best buy in the name of a bargain  no really  it can be fun   these aren t typically the models we d recommend if it didn t happen to be deals season  for our holiday gift guide  we skipped straight to   
	 an indiana man has a clerk in a clothing store to thank for the return of several thousand dollars that he nearly lost in his jacket pocket 
	 sam  club has made it easy in recent years to skip the cashier and purchase items via your own smartphone in store   except when it comes to alcohol  but sam  club announced this week tampa bay shoppers can skip the line  use the retailer  smartphone app and cash themselves out even if their order includes beer  wi

# Compare annoy results with Cosine similarity
- Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity

In [ ]:
from sentence_transformers import  util
# Here, we compute the recall of ANN compared to the exact results
correct_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=TOP_K)[0]
correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

In [ ]:
correct_hits_ids

{0, 535, 11143, 20510, 24802, 25544, 32078, 34074, 40905, 46408}

In [ ]:
annoy_hit_corpus=set(annoy_hit_corpus)
annoy_hit_corpus

{0, 5319, 11143, 13326, 15736, 26023, 30771, 36815, 46408, 49280}

In [ ]:

recall = len(annoy_hit_corpus.intersection(correct_hits_ids)) / len(correct_hits_ids)
print("recall: "+str(recall))

recall: 0.3
